In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import gzip

import numpy as np
from IPython import display

import time

from keras import backend as K
from keras.callbacks import Callback
from keras.models import Model, load_model, model_from_json
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, GlobalAveragePooling2D # Conv2D, Input, Flatten, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras.utils import HDF5Matrix
from keras.applications.vgg16 import VGG16

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

Using TensorFlow backend.


In [3]:
from utils.load_data import load_data
from utils.preprocess import DataGenerator
from utils.comparams import calculate_auc, auc

W0515 23:05:27.469206 139873690992768 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/utils/comparams.py:1: The name tf.local_variables_initializer is deprecated. Please use tf.compat.v1.local_variables_initializer instead.



### Data Generator

In [4]:
data_dir = './data/macenko'

In [5]:
x_test, y_test_true = load_data(data_dir, purpose='test', limit=None, val_limit=None, norm='macenko')

In [6]:
# indexes
test_id = np.arange(len(x_test))

# create a useful dictionary structures
partition = {}
partition['test'] = test_id
    
test_labels = {str(i) : y_test_true[i].flatten()[0] for i in test_id}

## VGG16 Predictions

In [7]:
# directory to save the best model
file_dir = './Model'
pred_file_dir = './Preds'
model_name = 'vgg16_model_10k'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [8]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

In [9]:
model = load_model(network_filepath)

W0512 12:09:15.459743 140127328833664 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0512 12:09:15.472865 140127328833664 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0512 12:09:15.488588 140127328833664 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0512 12:09:15.692600 140127328833664 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The

In [ ]:
preds = model.predict_generator(test_generator)

In [11]:
with open(os.path.join(pred_file_dir, 'preds_vgg16_10k.csv'), 'w') as f:
    f.write("case, prediction1, prediction2")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[0]) + ',' + str(p[1]) + '\n')

In [12]:
with open(os.path.join(pred_file_dir, 'preds_vgg16_10k_sub.csv'), 'w') as f:
    f.write("case, prediction\n")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[1]) + '\n')

##### Calculate AUC 

In [14]:
true_labels = np.array(y_test_true).flatten()

In [15]:
pred_labels = np.array([p[1] for p in preds])

In [27]:
calculate_auc(true_labels, pred_labels)

sklearn auc: 0.9129782979784034
tf auc: [0.9129074, 0.9129074]


## VGG19 Predictions

In [15]:
# directory to save the best model
file_dir = './Model'
pred_file_dir = './Preds'
model_name = 'vgg19_model_10K'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [17]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

In [28]:
from tensorflow import metrics, local_variables_initializer
from keras.backend import get_session
from sklearn.metrics import roc_auc_score as skroc
import tensorflow as tf

def auc(y_true, y_pred):
    auc = metrics.auc(y_true, y_pred)[1]
    get_session().run(local_variables_initializer())
    return auc

In [33]:
dependencies = {
    'auc': auc
}

In [34]:
model = load_model(network_filepath, custom_objects=dependencies)
# model = model_from_json(open(network_filepath).read())
# model.load_weights(os.path.join(os.path.dirname(network_filepath)))

W0512 03:24:42.024550 140039611105408 deprecation.py:323] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/tensorflow/python/ops/metrics_impl.py:808: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [35]:
preds = model.predict_generator(test_generator)

In [37]:
with open(os.path.join(pred_file_dir, 'preds_vgg19_10k.csv'), 'w') as f:
    f.write("case, prediction1, prediction2")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[0]) + ',' + str(p[1]) + '\n')

In [38]:
with open(os.path.join(pred_file_dir, 'preds_vgg19_10k_sub.csv'), 'w') as f:
    f.write("case, prediction\n")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[1]) + '\n')

##### Calculate AUC 

In [39]:
true_labels = np.array(y_test_true).flatten()

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels)

sklearn auc: 0.9086748213509703
tf auc: [0.90866363, 0.90866363]


## VGG19 Predictions with lr 0.001

In [40]:
del model

In [41]:
# directory to save the best model
file_dir = './Model'
pred_file_dir = './Preds'
model_name = 'vgg19_model_10K_lrr_0-001'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [42]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

In [43]:
from tensorflow import metrics, local_variables_initializer
from keras.backend import get_session
from sklearn.metrics import roc_auc_score as skroc
import tensorflow as tf

def auc(y_true, y_pred):
    auc = metrics.auc(y_true, y_pred)[1]
    get_session().run(local_variables_initializer())
    return auc

In [44]:
dependencies = {
    'auc': auc
}

In [45]:
model = load_model(network_filepath, custom_objects=dependencies)
# model = model_from_json(open(network_filepath).read())
# model.load_weights(os.path.join(os.path.dirname(network_filepath)))

In [47]:
preds = model.predict_generator(test_generator)

In [48]:
with open(os.path.join(pred_file_dir, 'preds_vgg19_10k_lrr_0-001.csv'), 'w') as f:
    f.write("case, prediction1, prediction2")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[0]) + ',' + str(p[1]) + '\n')

In [49]:
with open(os.path.join(pred_file_dir, 'preds_vgg19_10k_lrr_0-001_sub.csv'), 'w') as f:
    f.write("case, prediction\n")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[1]) + '\n')

##### Calculate AUC 

In [50]:
true_labels = np.array(y_test_true).flatten()

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels)

sklearn auc: 0.9062592216085712
tf auc: [0.90621924, 0.90621924]


# VGG19 with learningrate 0.0001

In [51]:
del model

In [57]:
# directory to save the best model
file_dir = './Model'
pred_file_dir = './Preds'
model_name = 'vgg19_model_10K_lrr_0.0001'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [53]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

In [54]:
from tensorflow import metrics, local_variables_initializer
from keras.backend import get_session
from sklearn.metrics import roc_auc_score as skroc
import tensorflow as tf

def auc(y_true, y_pred):
    auc = metrics.auc(y_true, y_pred)[1]
    get_session().run(local_variables_initializer())
    return auc

In [55]:
dependencies = {
    'auc': auc
}

In [58]:
model = load_model(network_filepath, custom_objects=dependencies)
# model = model_from_json(open(network_filepath).read())
# model.load_weights(os.path.join(os.path.dirname(network_filepath)))

In [61]:
preds = model.predict_generator(test_generator)

In [62]:
with open(os.path.join(pred_file_dir, 'preds_vgg19_10k_lrr_0-0001.csv'), 'w') as f:
    f.write("case, prediction1, prediction2")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[0]) + ',' + str(p[1]) + '\n')

In [63]:
with open(os.path.join(pred_file_dir, 'preds_vgg19_10k_lrr_0-0001_sub.csv'), 'w') as f:
    f.write("case, prediction\n")
    for i, p in enumerate(preds):
        f.write(str(i) + ',' + str(p[1]) + '\n')

##### Calculate AUC 

In [64]:
true_labels = np.array(y_test_true).flatten()

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels)

sklearn auc: 0.8990378027888102
tf auc: [0.89904654, 0.89904654]


# VGG19  Changable

In [12]:
# directory to save the best model
file_dir = './Model'
pred_file_dir = './Preds'
model_name = 'stain_norm_VGG19_model_10K'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [8]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

In [9]:
from tensorflow import metrics, local_variables_initializer
from keras.backend import get_session
from sklearn.metrics import roc_auc_score as skroc
import tensorflow as tf

def auc(y_true, y_pred):
    auc = metrics.auc(y_true, y_pred)[1]
    get_session().run(local_variables_initializer())
    return auc

In [10]:
dependencies = {
    'auc': auc
}

In [13]:
model = load_model(network_filepath, custom_objects=dependencies)
# model = model_from_json(open(network_filepath).read())
# model.load_weights(os.path.join(os.path.dirname(network_filepath)))

W0515 23:07:35.410932 139873690992768 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0515 23:07:35.423679 139873690992768 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0515 23:07:35.441139 139873690992768 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0515 23:07:35.668013 139873690992768 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The

In [14]:
preds = model.predict_generator(test_generator)

KeyboardInterrupt: 

In [ ]:
true_labels = np.array(y_test_true).flatten()

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels) # without beta lr=0.0001 learning rate reduction changed

In [ ]:
true_labels = np.array(y_test_true).flatten()

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels) # without beta lr=0.0001

In [13]:
true_labels = np.array(y_test_true).flatten()

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels) # 0.00007

sklearn auc: 0.8974235205864627
tf auc: [0.8974233, 0.8974233]
